In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
#import the lib tensorflow
import tensorflow as tf

In [3]:
import math

## Load Data

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
NUM_CLASS = 10
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE*IMAGE_SIZE

In [6]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_boolean("fake_data", False, 'If true, use fake data for unit testing')
flags.DEFINE_string("train_dir", 'data', 'Directory to put the data')
flags.DEFINE_integer("batch_size", 100, "batch size , must be divided evenly into the datasize")
flags.DEFINE_integer('hidden1_units', 128, 'number of units in hidden layer 1')
flags.DEFINE_integer('hidden2_units', 32, 'number of units in hidden layer 2')
flags.DEFINE_integer('max_train_steps', 200, 'number of the max training steps')

In [7]:
data_sets = input_data.read_data_sets(FLAGS.train_dir, FLAGS.fake_data)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


In [8]:
print(data_sets.train.labels[1:20])

[3 4 6 1 8 1 0 9 8 0 3 1 2 7 0 2 9 6 0]


## Build the model

### inference stage
Build the model:

args:

images : Image placeholder, 

hidden1_units : size of the first hidden layer

hidden2_units : size of the second hidden layer

In [9]:
def inference(images, hidden1_units, hidden2_units):
    #hidden1
    with tf.name_scope("hidden1"):
        weights = tf.Variable(tf.truncated_normal([IMAGE_PIXELS, hidden1_units], 
                                                  stddev=1.0/math.sqrt(float(IMAGE_PIXELS))),
                             name="weights")
        biases = tf.Variable(tf.zeros([hidden1_units]),
                            name="biases")
        hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
    
    #hidden2
    with tf.name_scope("hidden2"):
        weights = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units], 
                                                  stddev=1.0/math.sqrt(float(hidden1_units))),
                             name="weights")
        biases = tf.Variable(tf.zeros([hidden2_units]),
                            name="biases")
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    #Linear softmax layer
    with tf.name_scope("softmax_linear"):
        weights = tf.Variable(tf.truncated_normal([hidden2_units, NUM_CLASS], 
                                                  stddev=1.0/math.sqrt(hidden2_units)),
                             name="weights")
        biases = tf.Variable(tf.zeros([NUM_CLASS]),
                            name="biases")
        logits = tf.matmul(hidden2, weights) + biases
    return logits

## Loss Function

In [10]:
def loss_cal(logits, labels): 
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels, name="xentropy")
    loss = tf.reduce_mean(cross_entropy, name="xentropy_mean")
    return loss

## Evaluation the model

In [11]:
def evaluation(logits, labels):
  correct = tf.nn.in_top_k(logits, labels, 1)
  return tf.reduce_sum(tf.cast(correct, tf.int32))

def do_eval(sess, eval_correct, data_sets, image_placeholder, label_placeholder):
    true_count = 0
    steps_per_epoch = data_sets.num_examples // FLAGS.batch_size
    num_examples = steps_per_epoch * FLAGS.batch_size
    for step in xrange(steps_per_epoch):
        batch = data_sets.next_batch(FLAGS.batch_size, FLAGS.fake_data)
        true_count += sess.run(eval_correct, feed_dict={image_placeholder:batch[0], label_placeholder:batch[1]})
    precision = true_count / num_examples
    print('Precision: %f' % (precision))
    

# Train the model

In [15]:
# tell tensorflow that the model will build into the default Graph
with tf.Graph().as_default():
    # generate the placeholder for the imgaes and labels
    image_placeholder = tf.placeholder(tf.float32, shape=[FLAGS.batch_size, IMAGE_PIXELS])
    label_placeholder = tf.placeholder(tf.int32, shape=[FLAGS.batch_size])
    #build the graph from the inference model
    logits = inference(image_placeholder, FLAGS.hidden1_units, FLAGS.hidden2_units)
    loss = loss_cal(logits, label_placeholder)
    #
    tf.scalar_summary(loss.op.name, loss)
    print (loss.op.name)
    ##build the summary 
    summary_op = tf.merge_all_summaries()
    #
    sess = tf.Session()
    summary_writer = tf.train.SummaryWriter(FLAGS.train_dir, sess.graph)
    
    
    eval_correct = evaluation(logits, label_placeholder)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(loss)
    
    sess.run(tf.initialize_all_variables())
   
    for step in range(FLAGS.max_train_steps):
        batch = data_sets.train.next_batch(FLAGS.batch_size, FLAGS.fake_data)
        _ = sess.run([train_op,summary_op], feed_dict={image_placeholder: batch[0], label_placeholder: batch[1]})
        if step % 10 == 0:
            summary_str = sess.run(summary_op, feed_dict={image_placeholder: batch[0], label_placeholder: batch[1]})
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
    do_eval(sess, eval_correct, data_sets.test, image_placeholder, label_placeholder )

xentropy_mean
Precision: 0.477500
